## Pytorch NN API

Pytorch already includes everything we need to train a linear model in less than 10 lines of code!

In [1]:
%matplotlib inline
import random
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
import wandb
wandb.init(project='course') # specify the project of the current run

wandb: Currently logged in as: ingambe (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [3]:
true_w = torch.tensor([2, -3.4, 5, 6])
true_b = 2.4
features, labels = d2l.synthetic_data(true_w, true_b, 2000)

In [4]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 32
data_iter = load_array((features, labels), batch_size)

We can now iterate over minibtaches

In [5]:
next(iter(data_iter))

[tensor([[ 6.3635e-01,  4.8927e-01,  8.2714e-01,  1.3653e+00],
         [ 1.9924e+00,  1.2636e-01, -7.5565e-01, -9.1860e-01],
         [-7.7989e-01, -1.6402e-01, -9.0170e-02,  2.6409e-01],
         [ 1.7871e-01, -2.4488e-01, -7.8882e-01, -4.8697e-01],
         [-1.3268e+00,  2.5297e-01, -3.7057e-01, -1.8454e+00],
         [ 6.8182e-01,  3.8303e-01, -2.9347e-01, -8.0288e-02],
         [-6.5389e-01, -2.8404e-01,  1.0362e-01,  3.7658e-01],
         [-1.4634e+00, -6.4507e-01, -1.4562e+00,  7.2889e-01],
         [-2.3555e+00,  7.6512e-01, -2.0183e-01, -3.4557e-01],
         [ 5.1456e-01,  4.7775e-03,  2.7780e-01, -1.1815e+00],
         [ 4.9889e-01,  1.6713e-01,  1.9295e-01,  5.7369e-02],
         [-5.0471e-01, -1.2754e+00,  1.2549e+00,  9.9695e-01],
         [-3.0187e-01, -1.3290e+00, -8.3497e-01, -2.6912e-01],
         [-1.0125e+00,  6.7595e-01,  1.6548e-01, -6.0050e-01],
         [ 1.0572e+00, -1.2656e+00, -1.2197e+00, -3.6436e-01],
         [-1.8293e+00,  4.2128e-01,  3.9109e-01,  3.979

For standard operations, we can **use a framework's predefined layers,**
which allow us to focus on the layers used to construct the model
rather than having to focus on the implementation.

The `Sequential` class defines a container
for several layers that will be chained together.
Given input data, a `Sequential` instance passes it through
the first layer, in turn passing the output
as the second layer's input and so forth.

The layer is said to be *fully-connected*
because each of its inputs is connected to each of its outputs
by means of a matrix-vector multiplication.

In [6]:
# `nn` is an abbreviation for neural networks
from torch import nn

net = nn.Sequential(nn.Linear(4, 1))

We need to initialize the model parameters. By default Pytorch initialize the weight using an uniform distribution considering the size of the layer.

You should **always** initialize your layer

<center><img src="images/weights init.jpeg" /></center>

Gradient descent doesn't move you far away from the initial starting point

The literature is full of different weight initialization techniques

You can write yours:

In [7]:
net[0].weight.data.normal_(0, 0.01) # net[0] is the first layer
net[0].bias.data.fill_(0)
net[0].weight.data

tensor([[ 0.0090,  0.0003,  0.0039, -0.0010]])

99.9999% of the time you will use one from the literature: [See Pytorch init doc](https://pytorch.org/docs/stable/nn.init.html)

I recommend **Xavier normal**. It usually works well.
If you have time/ressource, you can try different init and pick the best ;)

In [8]:
def _weights_init(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_normal_(m.weight)
        m.bias.data.zero_()
        
net.apply(_weights_init)
net[0].weight.data

tensor([[ 1.1035, -0.7534,  0.5630, -0.2494]])

In [9]:
net[0].bias.data

tensor([0.])

Then we need to define the loss function we will use  
The `MSELoss` class computes the mean squared error, also known as squared $L_2$ norm  
By default, it returns the average loss over examples.

In [10]:
loss = nn.MSELoss()

Weights and Biases can kee an eye on your model, login the structure and the gradient

In [11]:
wandb.watch(net, log="all", criterion=loss, log_freq=1,  log_graph=(True)) #log frequency depend on your training

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Last piece of the puzzle, we need to define the optimizer  
When we (**instantiate an `SGD` instance,**) we will specify the parameters to optimize over (obtainable from our net via `net.parameters()`), with a dictionary of hyperparameters required by our optimization algorithm  
Minibatch stochastic gradient descent just requires that we set the value `lr`, which is set to 0.03 here.

In [12]:
optim = torch.optim.SGD(net.parameters(), lr=3e-2)

In [13]:
next(net.parameters())

Parameter containing:
tensor([[ 1.1035, -0.7534,  0.5630, -0.2494]], requires_grad=True)

Let's put everything together !

The training loop itself is strikingly similar to what we did when implementing everything from scratch.

For each minibatch, we go through the following ritual:

* Generate predictions by calling `net(X)` and calculate the loss `l` (the forward propagation).
* Calculate gradients by running the backpropagation.
* Update the model parameters by invoking our optimizer.

For good measure, we compute the loss after each epoch and print it to monitor progress.

In [14]:
num_epochs = 10
for epoch in range(num_epochs):
    for X, y in data_iter:
        y_hat = net(X)
        l = loss(y_hat, y)
        optim.zero_grad() # please don't forget!
        l.backward() # remember: You need to tell w.r.t what the gradient is computed
        optim.step() # do a step in the gradient direction
    with torch.no_grad():
        l = loss(net(features), labels) 
        print(f'epoch {epoch + 1}, loss {l.item()}')
        wandb.log({'loss': l.item()}, step=epoch)

epoch 1, loss 0.02376747503876686
epoch 2, loss 0.00010779940930660814
epoch 3, loss 9.956611756933853e-05
epoch 4, loss 0.00010070821008412167
epoch 5, loss 9.94472939055413e-05
epoch 6, loss 9.973853593692183e-05
epoch 7, loss 9.96062153717503e-05
epoch 8, loss 0.00010021294292528182
epoch 9, loss 9.957335714716464e-05
epoch 10, loss 9.98900577542372e-05


# ⚠️ NEVER FORGET TO ZERO_GRAD THE OPTIMIZER ⚠️

By default the optimizer accumulate the gradient!

If you don't set it back to 0, it will keep previous gradient and sum them!

If your model doesn't converge check this first!

Now let's compare the true parameters and the learned one:

In [15]:
w = net[0].weight.data
print('error in estimating w:', true_w - w.reshape(true_w.shape))
b = net[0].bias.data
print('error in estimating b:', true_b - b)

error in estimating w: tensor([-4.2677e-05,  6.3443e-04,  2.7037e-04, -8.1682e-04])
error in estimating b: tensor([0.0006])


We can save and load model to reuse them later

In [16]:
net.state_dict()

OrderedDict([('0.weight', tensor([[ 2.0000, -3.4006,  4.9997,  6.0008]])),
             ('0.bias', tensor([2.3994]))])

In [17]:
torch.save(net.state_dict(), 'my_model.pt')

This save model's parameters as a dictionnary, but **doesn't save the structure** of the neural network

In [18]:
new_model = nn.Sequential(nn.Linear(4, 1))
new_model.load_state_dict(torch.load('my_model.pt'))
new_model.state_dict()

OrderedDict([('0.weight', tensor([[ 2.0000, -3.4006,  4.9997,  6.0008]])),
             ('0.bias', tensor([2.3994]))])

wandb: Network error (ConnectionError), entering retry loop.
